In [5]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras import Input
from tensorflow.keras.models import Model

import numpy as np

In [3]:
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

In [7]:
def sampling(args):
    
    z_mean, _log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim), mean = 0.0, stddev = 1.0)
    
    return z_mean + K.exp(z_log_var) * epsilon

In [9]:
input_img = Input(shape = img_shape)

x = input_img
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu', strides = (2, 2))(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)

shape_before_flatening = K.int_shape(x)

x = Flatten()(x)
x = Dense(32, activation = 'relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

z = Lambda(sampling)([z_mean, z_log_var])

In [ ]:
decoder_input = Input(K.int_shape(z)[1:])

x = Dense(np.prod(shape_before_flatening[1:]), activation = 'relu')(decoder_input)
x = Reshape(shape_before_flatening[1:])(x)

x = Conv2DTranspose(32, 3, padding = 'same', activation = 'relu', strides = (2, 2))(x)

x = Conv2D(1, 3, padding = 'same', activation = 'sigmoid')(x)

decoder = Model